In [1]:
import pandas as pd

In [2]:
stories = pd.read_csv('../data/stories.csv')

/home/ubuntu/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (34,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273

In [3]:
stories = stories.drop(stories.filter(regex='Unnamed').columns, axis=1)

In [4]:
stories = stories.drop(columns=['license_type','hi_res_downloads', 'epub_downloads', 'donor_updated', 'attribution_text', 'credit_line', 'publisher',  'description', 'illustrator_name_updated', 'author_name_updated', 'translator_name_updated', 'status', 'reads', 'downloads_updated', 'published_date_updated', 'ancestry_updated', 'derivation_type_updated','reading_level_updated','category_name','tag_name'])

In [5]:
stories = stories[(stories['story_langugage']=="English") | (stories['story_langugage']=="Hindi")] 

In [6]:
stories.head()

,story_id,title,english_title,story_langugage,synopsis,content,story_original_title
0,1,वह हँस दिया,Veh Hans Diyaa,Hindi,क्‍या आपको दौड़ लगाने में मज़ा आता है? हमारी कहा...,Kitchenvadoekdiscl ClothesT-hemp&nbsp;&nbsp;&n...,वह हँस दिया
1,2,Smile Please!,Smile Please!,English,Follow the young deer as he races along with f...,A fawn was racing in the forest. He was ahead ...,वह हँस दिया
6,7,Fat King Thin Dog,Fat King Thin Dog,English,Run along with the Fat King after the Thin Dog!,This is a fat king. The fat king has a thin do...,Fat King Thin Dog
7,8,मोटा राजा दुबला कुत्ता,Mota Raja Dubla Kutta,Hindi,मोटे राजा के साथ दुबले कुत्ते के पीछे भागिये!\n,यह है मोटा राजा। मोटे राजा का है दुबला कुत्ता...,Fat King Thin Dog
15,16,"Vayu, the Wind","Vayu, the Wind",English,A nice story about this silent and always pres...,"Everytime I finish my hot, hot bath,My wet bod...",वारा


In [7]:
tags_list = ['<p>' ,'</p>' , '<p*>','\\n', '\\t', ' +'
             '<ul>','</ul>',
             '<li>','</li>',
             '<br>',
             '<strong>','</strong>',
             '<span*>','</span>',
             '<a href*>','</a>',
             '<em>','</em>']

for tag in tags_list:
    stories['content'].replace(to_replace=tag, value='', regex=True, inplace=True)

In [8]:
merged_stories = pd.merge(stories, stories[['content', 'title', 'synopsis', 'story_langugage']], right_on='title', left_on='story_original_title', suffixes=['','_original']).sort_index()

In [9]:
merged_stories = merged_stories.set_index('story_id')

In [10]:
merged_stories = merged_stories[merged_stories.pipe(lambda x: x['story_langugage'] != x['story_langugage_original']) & merged_stories.content.notnull()]

In [11]:
merged_stories_english = merged_stories[merged_stories['story_langugage_original'] == 'English']

In [12]:
anno_dataset = merged_stories_english[['title','content_original','content']]

In [13]:
anno_dataset.reset_index(inplace=True)

In [14]:
anno_dataset.replace(to_replace='&nbsp;', value='', regex=True, inplace=True)
anno_dataset.replace(to_replace='&nbsp', value='', regex=True, inplace=True)

/home/ubuntu/.local/lib/python3.6/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [15]:
anno_dataset.to_csv('../data/anno_stories.csv',index = False)